In [1]:
from pyspark import SparkConf
from pyspark.sql import SparkSession
import pandas as pd
import processing as procdata
from tweets.fileutils import read as read_json_tweets

#conf = SparkConf()
# conf.set('spark.jars.packages', 'org.apache.hadoop:hadoop-aws:3.2.0')
# conf.set('spark.hadoop.fs.s3a.aws.credentials.provider', 'com.amazonaws.auth.InstanceProfileCredentialsProvider')
#spark = SparkSession.builder.config(conf=conf).getOrCreate()

In [6]:
# SPARK


# PANDAS
file_geral_prouni = pd.DataFrame(pd.read_xml("pure_data/geralprouni.xml"))
file_addprouni = pd.read_json("pure_data/prouni.json")
file_edbasica = pd.read_csv("pure_data/microdados_ed_basica_2021.csv", sep=";", encoding="latin-1")
raw_tweets = read_json_tweets("pure_data/tweets.json")
file_tweets = pd.DataFrame(raw_tweets['data'])

/var/folders/90/4r9_jk_s59n2616g7y9snmn00000gn/T/ipykernel_2320/3792589478.py:7: DtypeWarning: Columns (27) have mixed types. Specify dtype option on import or set low_memory=False.
  file_edbasica = pd.read_csv("pure_data/microdados_ed_basica_2021.csv", sep=";", encoding="latin-1")


In [3]:
quantidade_bolsas = file_geral_prouni[['bolsa_integral_cotas', 'bolsa_integral_ampla', 'bolsa_parcial_cotas', 'bolsa_parcial_ampla']].sum(axis=1)
nota_bolsa = file_geral_prouni[['nota_integral_ampla', 'nota_integral_cotas', 'nota_parcial_ampla', 'nota_parcial_cotas']].mean(axis=1)

df_prouni = pd.DataFrame({
    'mensalidade': file_geral_prouni['mensalidade'],
    'estado': file_geral_prouni['uf_busca'],
    'municipio': file_geral_prouni['cidade_busca'].map(procdata.no_accents_lower),
    'curso': file_geral_prouni['nome'],
    'quantidade_bolsas': quantidade_bolsas,
    'nota_bolsa': nota_bolsa
})
df_prouni = df_prouni.dropna()
print(df_prouni.__len__(), "rows")
df_prouni.sample(n=5)

41447 rows


,mensalidade,estado,municipio,curso,quantidade_bolsas,nota_bolsa
21005,388.57,MG,uberaba,Gestão de Recursos Humanos,1.0,495.900000
1500,1628.40,CE,fortaleza,Fisioterapia,11.0,622.790000
19395,431.88,AM,manaus,Gestão da Qualidade,6.0,577.290000
3570,1283.00,RJ,petropolis,Nutrição,10.0,607.613333
30460,279.00,PR,campo largo,Gestão Pública,2.0,537.810000


In [4]:
df_addprouni = pd.DataFrame({
    'curso': file_addprouni['NOME_CURSO_BOLSA'],
    'estado': file_addprouni['SIGLA_UF_BENEFICIARIO_BOLSA'],
    'municipio': file_addprouni['MUNICIPIO_BENEFICIARIO_BOLSA'].map(procdata.no_accents_lower),
    'sexo': file_addprouni['SEXO_BENEFICIARIO_BOLSA'],
    'raca': file_addprouni['RACA_BENEFICIARIO_BOLSA'],
    'deficiente': file_addprouni['BENEFICIARIO_DEFICIENTE_FISICO'],
    'idade': file_addprouni['DT_NASCIMENTO_BENEFICIARIO']
})
df_addprouni = df_addprouni.dropna()
print(df_addprouni.__len__(), "rows")
df_addprouni.sample(n=5)

236636 rows


,curso,estado,municipio,sexo,raca,deficiente,idade
144476,Direito,PB,itabaiana,M,Parda,N,24/03/1997
130201,Psicologia,MG,juiz de fora,F,Parda,N,22/12/1998
93486,Gestão Da Tecnologia Da Informação,DF,brasilia,M,Parda,N,10/10/1987
173297,Logística,GO,cristalina,F,Preta,N,04/12/1995
138441,Processos Gerenciais,CE,caucaia,F,Parda,N,29/01/1979


In [5]:
df_edbasica = pd.DataFrame({
    'estado': file_edbasica['SG_UF'],
    'municipio': file_edbasica['NO_MUNICIPIO'].map(procdata.no_accents_lower),
    'esgoto_inexistente': file_edbasica['IN_ESGOTO_INEXISTENTE'],
    'energia_inexistente': file_edbasica['IN_ENERGIA_INEXISTENTE'],
    'agua_inexistente': file_edbasica['IN_AGUA_INEXISTENTE'],
    'acesso_internet': file_edbasica['IN_INTERNET'],
    'faz_exame_selecao': file_edbasica['IN_EXAME_SELECAO'],
    'especializada_deficientes': file_edbasica['IN_ESP'],
    'ensino_tecnico': file_edbasica['IN_PROF_TEC']
})
df_edbasica = df_edbasica.dropna()
print(df_edbasica.__len__(), "rows")
df_edbasica.sample(n=5)

178370 rows


,estado,municipio,esgoto_inexistente,energia_inexistente,agua_inexistente,acesso_internet,faz_exame_selecao,especializada_deficientes,ensino_tecnico
130107,ES,cariacica,0.0,0.0,0.0,1.0,1.0,1.0,1.0
86366,BA,barra do choca,0.0,0.0,0.0,1.0,0.0,1.0,0.0
11599,PA,ananindeua,0.0,0.0,0.0,1.0,0.0,1.0,0.0
179092,SP,taquaritinga,0.0,0.0,0.0,1.0,0.0,0.0,0.0
16911,PA,maraba,1.0,0.0,0.0,1.0,0.0,1.0,0.0


In [3]:
tweetsdf = pd.DataFrame({
    "sentiment": file_tweets["text"].map(procdata.analyze_tweet_sentiment),
    "positive": file_tweets["text"].map(lambda text: procdata.analyze_tweet_sentiment(text) > 0),
    "frequent": file_tweets["text"].map(procdata.analyze_tweet_frequency),
    "verified": file_tweets["author_id"].map(lambda u: procdata.analyze_tweet_verified(u, raw_tweets["users"])),
    "date": file_tweets["created_at"],
})

print(tweetsdf.__len__(), "rows")
tweetsdf.sample(n=5)

207 rows


,sentiment,positive,frequent,verified,date
47,-0.5993,False,"[(paguei, 1), (85, 1), (enem, 1), (p, 1), (con...",False,2022-05-22T13:47:32.000Z
48,-0.4404,False,"[(pessoas, 1), (sabiam, 1), (fiquei, 1), (quar...",False,2022-05-22T14:06:14.000Z
27,-0.2960,False,"[(sétimo, 1), (dia, 1), (faltando, 1), (faculd...",False,2022-05-23T12:31:37.000Z
204,-0.1531,False,"[(enem, 2), (olho, 1), (prazo, 1), (inscrições...",False,2022-05-17T15:00:23.000Z
82,0.0000,False,"[(comemorei, 1), (pq, 1), (consegui, 1), (bols...",False,2022-05-21T00:04:16.000Z


In [7]:
df_addprouni = df_addprouni.drop_duplicates(subset=['estado', 'curso', 'municipio'])
df_edbasica = df_edbasica.drop_duplicates(subset=['estado', 'municipio'])
df = df_prouni.merge(df_addprouni, on=['estado', 'curso', 'municipio'], how='left')
df = df.merge(df_edbasica, on=['estado', 'municipio'], how='left')
df = df.dropna()

print(df.__len__(), "rows")
df.sample(n=5)

25856 rows


,mensalidade,estado,municipio,curso,quantidade_bolsas,nota_bolsa,sexo,raca,deficiente,idade,esgoto_inexistente,energia_inexistente,agua_inexistente,acesso_internet,faz_exame_selecao,especializada_deficientes,ensino_tecnico
9294,884.29,RR,boa vista,Nutrição,3.0,602.04,F,Parda,N,13/09/1973,0.0,0.0,0.0,1.0,1.0,1.0,1.0
4619,1191.00,SP,campinas,Psicologia,5.0,637.19,F,Parda,N,09/04/1999,0.0,0.0,0.0,1.0,0.0,1.0,0.0
41339,149.00,SP,bauru,Gestão Comercial,1.0,450.00,F,Branca,N,03/05/1999,0.0,0.0,0.0,1.0,0.0,1.0,0.0
41016,177.23,RN,natal,Pedagogia,11.0,573.14,M,Parda,N,30/01/1974,0.0,0.0,0.0,1.0,0.0,1.0,0.0
38151,237.00,MA,balsas,Geografia,1.0,559.84,F,Preta,N,08/05/1997,0.0,0.0,0.0,1.0,0.0,1.0,0.0


In [4]:
# SPARK
# df.write.csv('processed_data/prouni.csv')

# PANDAS
df.to_csv('processed_data/g2-prouni-preprocessado.csv')
tweetsdf.to_csv('processed_data/g2-tweets-preprocessado.csv')